In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
# 定义主要美股指数的代码
indices = {
    'QQQ': 'QQQ',              # 纳斯达克100指数ETF
    'SPY': 'SPY',              # 标普500指数ETF
    'DIA': 'DIA',              # 道琼斯工业平均指数ETF
    'IWM': 'IWM',              # 罗素2000小盘股指数ETF
    'VTI': 'VTI',              # 全美股票市场ETF
    'XLK': 'XLK',              # 科技板块ETF
    'XLF': 'XLF',              # 金融板块ETF
    'XLE': 'XLE',              # 能源板块ETF
    'XLV': 'XLV',              # 医疗保健ETF
    'TQQQ': 'TQQQ',            # 纳斯达克100指数3倍杠杆ETF
    '^IXIC': '^IXIC',          # 纳斯达克综合指数
    '^GSPC': '^GSPC',          # 标普500指数
    '^DJI': '^DJI',            # 道琼斯工业平均指数
    '^RUT': '^RUT',            # 罗素2000指数
    '^VIX': '^VIX',            # VIX恐慌指数
}

etfs = {
    "ARKK": "创新科技",
    "ARKW": "下一代互联网",
    "ARKG": "基因组革命",
    "ARKF": "金融科技",
    "ARKX": "太空探索",
    "ICLN": "清洁能源",
    "PBW": "清洁能源",
    "TAN": "太阳能",
    "FAN": "风能",
    "HERO": "电子竞技",
    "ESPO": "电子竞技",
    "NERD": "电子竞技",
    "SOCL": "社交媒体",
    "FINX": "金融科技",
    "FIVG": "5G网络",
    "AIQ": "人工智能",
    "ROBO": "机器人与自动化",
    "BOTZ": "机器人与人工智能",
    "HACK": "网络安全",
    "CIBR": "网络安全",
    "SKYY": "云计算",
    "WCLD": "云计算",
    "CLOU": "云计算",
    "SNSR": "物联网",
    "IBUY": "电子商务",
    "ONLN": "在线零售",
    "AWAY": "旅游科技",
    "BETZ": "体育博彩",
    "MJ": "大麻",
    "YOLO": "大麻",
    "POTX": "大麻",
    "PSY": "精神药物",
    "PHO": "水资源",
    "FIW": "水资源",
    "JETS": "航空",
    "IYT": "交通运输",
    "IHF": "医疗保健设备",
    "XBI": "生物科技",
    "IBB": "生物科技",
    "SMH": "半导体",
    "SOXX": "半导体",
    "REMX": "稀土金属",
    "LIT": "锂电池",
    "BATT": "电池技术",
    "GRID": "智能电网",
    "CNRG": "能源变革"
}

In [13]:
def get_index_info(ticker_symbol):
    """获取指数或ETF的基本信息"""
    ticker = yf.Ticker(ticker_symbol)
    info = ticker.info
    print(f"ticker: {ticker_symbol}, info keys: \n")
    for key, value in info.items():
        print(f"{key}: {value}")
    # 创建存储信息的字典
    index_info = {}
    
    # 提取关键信息（基于是ETF还是指数可能有所不同）
    try:
        # 通用信息
        index_info['shortName'] = info.get('shortName', info.get('longName', 'N/A'))
        index_info['exchange'] = info.get('exchange', 'N/A')
        index_info['currency'] = info.get('currency', 'N/A')
        
        # ETF特有信息
        if 'fundFamily' in info:  # 判断是否为ETF
            index_info['type'] = 'ETF'
            # 所属基金公司
            index_info['fundFamily'] = info.get('fundFamily', 'N/A')
            # 净资产
            index_info['totalAssets'] = info.get('totalAssets', 'N/A')
            # 费率
            index_info['expenseRatio'] = info.get('expenseRatio', 'N/A')
            # 派息率
            index_info['yield'] = info.get('yield', 'N/A')
            # 成立日期
            index_info['fundInceptionDate'] = info.get('fundInceptionDate', 'N/A')
            # 52周高点
            index_info['fiftyTwoWeekHigh'] = info.get('fiftyTwoWeekHigh', 'N/A')
            # 52周低点
            index_info['fiftyTwoWeekLow'] = info.get('fiftyTwoWeekLow', 'N/A')
            # 日均交易量
            index_info['averageVolume'] = info.get('averageVolume', 'N/A')
        else:  # 指数
            index_info['type'] = 'indices'
            # 当前值
            index_info['regularMarketPrice'] = info.get('regularMarketPrice', 'N/A')
            # 52周高点
            index_info['fiftyTwoWeekHigh'] = info.get('fiftyTwoWeekHigh', 'N/A')
            # 52周低点
            index_info['fiftyTwoWeekLow'] = info.get('fiftyTwoWeekLow', 'N/A')
            
    except Exception as e:
        print(f"获取 {ticker_symbol} 信息时出错: {e}")
    
    return index_info

In [4]:
def get_index_performance(ticker_symbol, period='1mo'):
    """获取指数过去一段时间的表现"""
    try:
        # 获取历史数据
        data = yf.download(ticker_symbol, period=period)
        
        # 如果数据为空，返回None
        if data.empty:
            return None
        
        # 计算各种表现指标
        performance = {}
        
        # 确保数据至少有两行
        if len(data) > 1:
            # 最新收盘价
            latest_close = data['Close'].iloc[-1]
            performance['最新收盘价'] = latest_close
            print(f"lastest_close: {latest_close}")
            
            # 今年至今回报率
            start_of_year = datetime(data.index[-1].year, 1, 1)
            try:
                ytd_start_price = data.loc[data.index >= start_of_year, 'Close'].iloc[0]
                performance['今年至今回报'] = (latest_close / ytd_start_price - 1) * 100
            except:
                performance['今年至今回报'] = 'N/A'
            
            # 1个月回报率
            try:
                one_month_ago = data.index[-1] - timedelta(days=30)
                month_price = data.loc[data.index >= one_month_ago, 'Close'].iloc[0]
                performance['1个月回报'] = (latest_close / month_price - 1) * 100
            except:
                performance['1个月回报'] = 'N/A'
            
            # 3个月回报率
            try:
                three_months_ago = data.index[-1] - timedelta(days=90)
                three_month_price = data.loc[data.index >= three_months_ago, 'Close'].iloc[0]
                performance['3个月回报'] = (latest_close / three_month_price - 1) * 100
            except:
                performance['3个月回报'] = 'N/A'
            
            # 6个月回报率
            try:
                six_months_ago = data.index[-1] - timedelta(days=180)
                six_month_price = data.loc[data.index >= six_months_ago, 'Close'].iloc[0]
                performance['6个月回报'] = (latest_close / six_month_price - 1) * 100
            except:
                performance['6个月回报'] = 'N/A'
            
            # 1年回报率
            try:
                one_year_ago = data.index[-1] - timedelta(days=365)
                year_price = data.loc[data.index >= one_year_ago, 'Close'].iloc[0]
                performance['1年回报'] = (latest_close / year_price - 1) * 100
            except:
                performance['1年回报'] = 'N/A'
            
            # 3年回报率
            try:
                three_years_ago = data.index[-1] - timedelta(days=3*365)
                three_year_price = data.loc[data.index >= three_years_ago, 'Close'].iloc[0]
                performance['3年回报'] = (latest_close / three_year_price - 1) * 100
            except:
                performance['3年回报'] = 'N/A'
            
            # 5年回报率
            try:
                five_years_ago = data.index[-1] - timedelta(days=5*365)
                five_year_price = data.loc[data.index >= five_years_ago, 'Close'].iloc[0]
                performance['5年回报'] = (latest_close / five_year_price - 1) * 100
            except:
                performance['5年回报'] = 'N/A'
            
            # 计算波动率(标准差)
            performance['年化波动率'] = data['Close'].pct_change().std() * (252 ** 0.5) * 100  # 252个交易日
            # 计算最大回撤
            cumulative_returns = (1 + data['Close'].pct_change().fillna(0)).cumprod()
            rolling_max = cumulative_returns.expanding().max()
            drawdown = (cumulative_returns / rolling_max - 1)
            performance['最大回撤'] = drawdown.min() * 100
        
        return performance
    
    except Exception as e:
        print(f"计算 {ticker_symbol} 表现时出错: {e}")
        return None

#### 获取etf的所有/topn 持仓股票(该方法无效,无结果)

In [7]:
def get_etf_holdings(ticker_symbol, top_n=10):
    """获取ETF的前N大持仓，如果可用"""
    ticker = yf.Ticker(ticker_symbol)
    
    try:
        holdings = ticker.holdings
        if holdings is not None and not isinstance(holdings, str):
            top_holdings = []
            
            # 尝试提取持仓信息
            if 'holdings' in holdings and 'holdings_names' in holdings and 'holdings_weights' in holdings:
                for i in range(min(top_n, len(holdings['holdings']))):
                    if i < len(holdings['holdings_names']) and i < len(holdings['holdings_weights']):
                        top_holdings.append({
                            'Symbol': holdings['holdings'][i],
                            'Name': holdings['holdings_names'][i],
                            'Weight': holdings['holdings_weights'][i]
                        })
            
            return top_holdings
        else:
            return []
    except:
        return []

In [8]:
result = get_etf_holdings('QQQ')
print(f"etf topk holdings: {result}")

etf topk holdings: []


In [28]:
def analyze_index_correlations(indices_list, period='1y'):
    """分析指数之间的相关性"""
    try:
        # 获取所有指数的历史数据
        data = yf.download(list(indices_list.values()), period=period)['Close']
        
        # 计算每日回报率
        returns = data.pct_change().dropna()
        
        # 计算相关性矩阵
        correlation_matrix = returns.corr()
        
        return correlation_matrix
    except Exception as e:
        print(f"计算相关性时出错: {e}")
        return None

In [29]:
def plot_index_comparison(indices_list, period='1y'):
    """绘制指数比较图表"""
    try:
        # 获取所有指数的历史数据
        data = yf.download(list(indices_list.values()), period=period)['Close']
        
        # 将所有指数归一化为基准100
        normalized_data = data.div(data.iloc[0]).mul(100)
        
        # 绘制比较图
        plt.figure(figsize=(12, 6))
        for symbol in normalized_data.columns:
            label = next((name for name, ticker in indices_list.items() if ticker == symbol), symbol)
            plt.plot(normalized_data.index, normalized_data[symbol], label=label)
        
        plt.title('指数表现比较(基准化为100)')
        plt.xlabel('日期')
        plt.ylabel('指数(初始值=100)')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        
        # 保存图表
        plt.savefig('index_comparison.png')
        
        return 'index_comparison.png'
    except Exception as e:
        print(f"绘制比较图时出错: {e}")
        return None


In [14]:
# 获取QQQ的详细信息
qqq_info = get_index_info('QQQ')
print("\nQQQ基本信息:")
for key, value in qqq_info.items():
    print(f"{key}: {value}")

# 获取QQQ表现
# qqq_performance = get_index_performance('QQQ')
# print("\nQQQ表现:")
# for key, value in qqq_performance.items():
#     if isinstance(value, float):
#         print(f"{key}: {value:.2f}%")
#     else:
#         print(f"{key}: {value}")
#     print("\n")

ticker: QQQ, info keys: 

longBusinessSummary: To maintain the correspondence between the composition and weights of the securities in the trust (the "securities") and the stocks in the NASDAQ-100 Index®, the adviser adjusts the securities from time to time to conform to periodic changes in the identity and/or relative weights of index securities. The composition and weighting of the securities portion of a portfolio deposit are also adjusted to conform to changes in the index.
companyOfficers: []
executiveTeam: []
maxAge: 86400
priceHint: 2
previousClose: 483.3
open: 488.54
dayLow: 484.17
dayHigh: 492.7
regularMarketPreviousClose: 483.3
regularMarketOpen: 488.54
regularMarketDayLow: 484.17
regularMarketDayHigh: 492.7
trailingPE: 30.62097
volume: 38867968
regularMarketVolume: 38867968
averageVolume: 48822867
averageVolume10days: 37736800
averageDailyVolume10Day: 37736800
bid: 487.74
ask: 488.21
bidSize: 65
askSize: 1
yield: 0.0063
totalAssets: 299143036928
fiftyTwoWeekLow: 402.39
fifty

In [19]:
# 获取etf ARKK的详细信息
qqq_info = get_index_info('ARKK')
print("\nARKK基本信息:")
for key, value in qqq_info.items():
    print(f"{key}: {value}")

# 获取QQQ表现
qqq_performance = get_index_performance('ARKK')
print("\nARKK表现:")
for key, value in qqq_performance.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}%")
    else:
        print(f"{key}: {value}")


ARKK基本信息:
名称: ARK Innovation ETF
交易所: BTS
货币: USD
类型: ETF
基金公司: ARK ETF Trust
净资产: 4958552064
费率: N/A
派息率: 0.0
成立日期: 1414713600
52周高点: 68.43
52周低点: 36.85
日均交易量: 12939216


[*********************100%***********************]  1 of 1 completed


ARKK表现:
最新收盘价: Ticker
ARKK    50.845001
Name: 2025-05-05 00:00:00, dtype: float64
今年至今回报: Ticker
ARKK   -10.625766
dtype: float64
1个月回报: Ticker
ARKK    20.257811
dtype: float64
3个月回报: Ticker
ARKK   -19.447082
dtype: float64
6个月回报: Ticker
ARKK   -2.483694
dtype: float64
1年回报: Ticker
ARKK    9.344089
dtype: float64
3年回报: Ticker
ARKK    12.358184
dtype: float64
5年回报: Ticker
ARKK   -5.079837
dtype: float64
年化波动率: Ticker
ARKK    47.139232
dtype: float64
最大回撤: Ticker
ARKK   -80.914108
dtype: float64
